In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append("..")
from simdb.databaseModel import *
from simdb.databaseAPI import *

In [3]:
# db_path = 'test_WORKS.db'
db_path = '../examples/setup_databases/andrej_raw.db'

In [4]:
from sqlalchemy import or_, and_

## For migration to databaseAPI:

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [ ]:
class DatabaseConnection(object):
    
    def __init__(self, db_path):
        self.db_path = db_path
        self.session = None
        

    def __enter__(self):
        # make a database connection and return it
        if not os.path.exists(self.db_path):
            raise "%s does not exist."
        
        engine = create_engine('sqlite:///{}'.format(self.db_path))
        Session = sessionmaker()
        self.session = Session(bind=engine)
        
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.session.close()

In [ ]:

class SQLAlchemyDBConnection(object):
    """SQLAlchemy database connection"""

    def __init__(self, connection_string):
        self.connection_string = connection_string
        self.session = None

    def __enter__(self):
        engine = create_engine(self.connection_string)
        Session = sessionmaker()
        self.session = Session(bind=engine)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.session.close()

In [ ]:
with DatabaseConnection(db_path) as db:
    print db.session.query(Keywords)

In [ ]:
q = s.session.query(Keywords)
[e.name for e in q.filter(Keywords.value != None).all()]

In [ ]:
s.close()

In [ ]:
s.add(Main())

In [ ]:
s.commit()

In [ ]:
s.close()

## HowTo

### Load Entry Table

In [ ]:
table = getEntryTable(db_path, load_keys=True, load_tags=True) # load whole table

In [13]:
table = get_entry_table(db_path)

/usr/local/lib/python2.7/dist-packages/sqlalchemy/sql/base.py:490: SAWarning: Column 'id' on table <sqlalchemy.sql.selectable.Select at 0x7f6ea5d91c10; Select object> being replaced by Column('id', Integer(), table=<Select object>, primary_key=True, nullable=False), which has the same key.  Consider use_labels for select() statements.
  self[column.key] = column


In [17]:
table

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

### Obtain data for certain entry

In [5]:
entry_id = "2017_08_01_2ub_k27_02_06"

In [11]:
details = get_entry_details(db_path, entry_id)
details

{'added_on': datetime.datetime(2018, 11, 24, 0, 22, 20, 543810),
 'children': [],
 'created_on': datetime.datetime(2017, 8, 26, 15, 48, 16, 765571),
 'description': u'',
 'entry_id': u'2017_08_01_2ub_k27_02_06',
 'groups': [u'Group 2'],
 'id': 117,
 'keywords': {u'chainlen': u'2',
  u'ff': u'MARTINI',
  u'linker': u'k27',
  u'number': u'06',
  u'protein': u'UBQ',
  u'structure': u'02'},
 'meta': {u'barostat': {u'p_compressibility': u'0.0003',
   u'p_coupling': u'iso',
   u'p_rel': u'12',
   u'p_target': u'1',
   u'type': u'Parrinello-Rahman'},
  u'thermostat': {u'T_rel': u'1', u'T_target': u'300', u'type': u'v-rescale'}},
 'owner': u'andrejb',
 'parents': [],
 'path': u'/home/andrejb/Research/SIMS/2017_08_01_2ub_k27_02_06',
 'tags': [],
 'type': None,
 'updated_on': datetime.datetime(2018, 11, 24, 0, 22, 20, 543844),
 'url': None}

### Select entries

In [ ]:
table[selectByTag(table, "test")] # select by tag

In [ ]:
table[selectByKeyword(table, "ff", "MARTINI")] # select by keyword

In [ ]:
# combinations of selections
ff       = selectByKeyword(table, "ff", "MARTINI")
chainlen = selectByKeyword(table, "chainlen", "1")
tags     = selectByTag(table, "test2")

table[ff & chainlen & tags]

In [ ]:
# another example
ff       = selectByKeyword(table, "ff", "MARTINI")
chainlen = selectByKeyword(table, "chainlen", "2")
linker1  = selectByKeyword(table, "linker", "k48")
linker2  = selectByKeyword(table, "linker", "k63")
number   = selectByKeyword(table, "number", "01")

table[ff & chainlen & (linker1 | linker2) & ~number]

### Working with groups

In [ ]:
# add some entries to a group
sel = table[selectByKeyword(table, "linker", "k48")]
add_to_group(db_path, sel.entry_id, "Group 1")

# a second group
sel = table[selectByKeyword(table, "ff", "MARTINI")]
add_to_group(db_path, sel.entry_id, "Group 2")

In [ ]:
# get entries of group 1
entry_group_table1 = get_entry_table(db_path, group_names=["Group 1"], columns=["entry_id", "path"])
print("{} entries loaded.".format(entry_group_table1.shape[0]))

# get entries of group 2
entry_group_table2 = get_entry_table(db_path, group_names=["Group 2"], columns=["entry_id", "path"])
print("{} entries loaded.".format(entry_group_table2.shape[0]))

In [ ]:
# get entries from both groups
# some entries might be in both groups
merged_table = get_entry_table(db_path, group_names=["Group 1", "Group 2"], columns=["entry_id", "path"])
print("Merge results in {} entries.".format(merged_table.shape[0]))

In [ ]:
# remove some etnries from Group 1
sel = table[selectByKeyword(table, "ff", "MARTINI")]
remove_from_group(db_path, sel.entry_id, "Group 1")

### Usefull for backend / viewer

In [ ]:
# open database (in SQLAlchemy we open a session)
# s = openDatabase(db_path) 
# one should not keep the DB open

# better:
with openDatabase(db_path) as db:
    print db
    

In [ ]:
get_tags(db_path)  # get all tags which are used in DB

In [ ]:
keywords = get_keywords(db_path)  # get all keyword with the values they have
pd.Series(keywords) 

In [ ]:
get_groups(db_path)

### Running functions

## Deva

* Add tags/keywords to all simulations in pandas table
* Add simulations to DB from pandas table